In [ ]:
!pip install openpyxl mlflow pandas scikit-learn==1.5.2 oracledb python-dotenv tensorflow scikeras xgboost lightgbm seaborn --proxy http://172.25.156.90:3128

In [ ]:
from db_utils import connect_to_oracle, get_data_for_anomaly_type
from pipeline_utils import create_pipelines, evaluate_pipeline, evaluate_pipeline_oodd, create_pipelines_for_probs, evaluate_pipeline_from_probs
from sklearn.model_selection import train_test_split
import sys
sys.path.append('./src/oodd_detectors')
sys.path.append('./src/preprocessors')
from target_preprocessor import create_target_pipeline
import mlflow
from continuous_OODD import OODDContinuousModel
from categorical_OODD import OODDCategoricalModel, OODDSmoothedCategoricalModel
from counter_OODD import CountBasedClassifier, FallbackCountClassifier
from integer_counter_OODD import ContinuousCountBasedClassifier, FallbackContinuousCountClassifier
from plot_utils import plot_target_distribution
import pandas as pd
from plot_utils import plot_target_distribution
from db_utils import get_data_for_anomaly_type
# needs scikit-learn version 1.5.2 (default 1.6.1 throws 'super' object has no attribute '__sklearn_tags__'.
# from scikeras.wrappers import KerasClassifier
connection, cursor = connect_to_oracle()

In [ ]:
anomaly_type = "TEMPERATURE"
raw_df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=3000000)
plot_target_distribution(raw_df, 'TEMPERATURE')
plot_target_distribution(raw_df, 'TEMP_TO')
plot_target_distribution(raw_df, 'TEMP_FROM')
plot_target_distribution(raw_df, 'HEATING_TYPE')
# plot_target_distribution(raw_df, 'HEATING_NOTE')
print(raw_df)

In [ ]:
from typing import List, Tuple
def count_none_and_complete_rows(
    df: pd.DataFrame,
    cols: List[str]
) -> Tuple[pd.Series, int]:
    """
    For the given DataFrame and list of columns:
      - Returns a Series of % null (None/NaN) per column.
      - Returns the % of rows with no nulls in those columns.
    """
    # validate columns
    missing = set(cols) - set(df.columns)
    if missing:
        raise KeyError(f"Unknown columns: {missing!r}")

    # % null per column
    null_pct = df[cols].isnull().mean() * 100

    # % complete rows across those columns
    complete_pct = df[cols].notnull().all(axis=1).mean() * 100

    return null_pct, complete_pct

null_counts, complete_row = count_none_and_complete_rows(raw_df,['TEMPERATURE','TEMP_TO','TEMP_FROM'])
print(null_counts)
print(complete_row)

In [ ]:
import pandas as pd
from typing import Optional, List, Dict

def most_common_values_pct(
    df: pd.DataFrame,
    cols: Optional[List[str]] = None,
    top_n: int = 1
) -> Dict[str, pd.Series]:
    """
    For each specified column (or all if cols is None), 
    return its top_n most frequent values (including NaNs) 
    as percentages of the total rows.
    """
    if cols is None:
        cols = list(df.columns)
    missing = set(cols) - set(df.columns)
    if missing:
        raise KeyError(f"Unknown columns: {missing!r}")

    result: Dict[str, pd.Series] = {}
    total = len(df)
    for col in cols:
        pct = (
            df[col]
            .value_counts(dropna=False, normalize=True)
            .mul(100)
            .head(top_n)
        )
        result[col] = pct
    return result
top_pct = most_common_values_pct(raw_df, top_n=4)
for col, series in top_pct.items():
    print(f"Column {col!r}:")
    print(series.round(2))
    
    print()

In [ ]:
anomaly_type = "HEATING_TYPE"
bht_df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(bht_df, y_col, "Hea")
bht_df[y_col] = bht_df[y_col].map(
                {None: 0.0, ' ': 0.0, 'X': 1.0}).fillna(bht_df[y_col])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
bht_df[y_col] = le.fit_transform(bht_df[y_col])
plot_target_distribution(bht_df, y_col, "Customs number")

In [ ]:
import pandas as pd
from typing import List, Tuple

def pct_blank_columns(
    df: pd.DataFrame,
    cols: List[str]
) -> Tuple[float, float]:
    """
    For the given DataFrame and list of columns, returns:
      - percent_all_blank: % of rows where ALL specified cols are None or empty string.
      - percent_any_blank: % of rows where ANY specified col is None or empty string.
    """
    missing = set(cols) - set(df.columns)
    if missing:
        raise KeyError(f"Unknown columns: {missing!r}")

    stripped = df[cols].astype(str).apply(lambda s: s.str.strip() == "")


    mask = df[cols].isnull() | (df[cols] == "") | (df[cols] == " ") | stripped
    percent_all_blank = mask.all(axis=1).mean() * 100
    percent_any_blank = mask.any(axis=1).mean() * 100
    return percent_all_blank, percent_any_blank

all_pct, any_pct = pct_blank_columns(raw_df, ['TEMPERATURE','TEMP_TO','TEMP_FROM'])
print(f"All blank: {all_pct:.2f}%")  # e.g. 50.0%
print(f"Any blank: {any_pct:.2f}%")  # e.g. 75.0%


In [ ]:
anomaly_type = "BHT"
bht_df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(bht_df, y_col, "Customs number raw")
bht_df[y_col] = bht_df[y_col].map(
                {None: 0.0, ' ': 0.0, 'X': 1.0}).fillna(bht_df[y_col])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
bht_df[y_col] = le.fit_transform(bht_df[y_col])
plot_target_distribution(bht_df, y_col, "Customs number")

In [ ]:
anomaly_type = "HEATING_TYPE"
bht_df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(bht_df, y_col, "Heating Type raw")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
bht_df[y_col] = le.fit_transform(bht_df[y_col])
plot_target_distribution(bht_df, y_col, "Heating Type")

In [ ]:
anomaly_type = "IM_TK_HEAT"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(df, y_col, "Has heating raw")
df[y_col] = df[y_col].map(
                {None: 0.0, ' ': 0.0, 'X': 1.0}).fillna(df[y_col])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[y_col] = le.fit_transform(df[y_col])
plot_target_distribution(df, y_col, "Has heating")

In [ ]:
anomaly_type = "TEMPERATURE"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(df, y_col, "TEMPERATURE raw")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[y_col] = le.fit_transform(df[y_col])
plot_target_distribution(df, y_col, "TEMPERATURE")

In [ ]:
anomaly_type = "T_CH_TYP_NA"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(df, y_col, "Chassis type import raw")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[y_col] = le.fit_transform(df[y_col])
plot_target_distribution(df, y_col, "Chassis type import")

In [ ]:
anomaly_type = "T_CH_TYP_VY"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(df, y_col, "Chassis type export raw")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[y_col] = le.fit_transform(df[y_col])
plot_target_distribution(df, y_col, "Chassis type export")

In [ ]:
anomaly_type = "EX_SP_REF"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(df, y_col, "Additional reference export raw")
df[y_col] = df[y_col].astype(str)
df[y_col] = df[y_col].apply(
    lambda x: 0 if pd.isna(x) or str(x).strip() == '' else 1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[y_col] = le.fit_transform(df[y_col])
plot_target_distribution(df, y_col, "Additional reference export")

In [ ]:
anomaly_type = "IM_SP_REF"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000000)
plot_target_distribution(df, y_col, "Additional reference import raw")
df[y_col] = df[y_col].astype(str)
df[y_col] = df[y_col].apply(
    lambda x: 0 if pd.isna(x) or str(x).strip() == '' else 1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[y_col] = le.fit_transform(df[y_col])
plot_target_distribution(df, y_col, "Additional reference import")